# Lecture 17 – Data 100, Spring 2024

Data 100, Spring 2024

[Acknowledgments Page](https://ds100.org/sp24/acks/)

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px


---

## A Random Variable $X$

Our probability distribution of $X$, shown as a table:

In [ ]:
# Our random variable X
dist_df = pd.DataFrame({"x": [3, 4, 6, 8],
                        "P(X = x)": [0.1, 0.2, 0.4, 0.3]})
dist_df

In [ ]:
fig = px.bar(dist_df, x="x", y="P(X = x)", title="Distribution of X")
# fig.write_image("distX.png", "png",scale=2)
fig

Let's use this probability distribution to generate a table of $X(s)$, i.e., random variable values for many many samples.

In [ ]:
N = 80000
samples = np.random.choice(
    dist_df["x"], # Draw from these choiecs
    size=N, # This many times
    p=dist_df["P(X = x)"]) # According to this distribution

sim_df = pd.DataFrame({"X(s)": samples})
sim_df

<br/><br/>
Let's check how well this simulated sample matches our probability distribution!

In [ ]:
fig = px.histogram(sim_df, x="X(s)", title="Empirical distribution of X", 
                   histnorm="probability")
# fig.write_image("empirical_dist.png", "png",scale=2)
fig

In [ ]:
print("Simulated E[X]:", sim_df['X(s)'].mean())
print("Simulated Var[X]:", sim_df['X(s)'].var())

In [ ]:
E_x = dist_df["x"] @ dist_df["P(X = x)"]
print("E[X]:",E_x)

In [ ]:
Var_x = dist_df["x"]**2 @ dist_df["P(X = x)"] - E_x**2
print("Var[X]:", Var_x)

<br/><br/>

---

# Sum of 2 Dice Rolls

Here's the distribution of a single die roll:

In [ ]:
roll_df = pd.DataFrame({"x": [1, 2, 3, 4, 5, 6],
                        "P(X = x)": np.ones(6)/6})
roll_df

In [ ]:
fig = px.bar(roll_df, x="x", y="P(X = x)", title="Distribution of X")
# fig.write_image("die.png", "png",scale=2)
fig

Let $X_1, X_2$ are the outcomes of two dice rolls. Note $X_1$ and $X_2$ are i.i.d. (independent and identically distributed).

Below I call a helper function `simulate_iid_df`, which simulates an 80,000-row table of $X_1, X_2$ values. It uses `np.random.choice(arr, size, p)` [link](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html) where `arr` is the array the values and `p` is the probability associated with choosing each value. If you're interested in the implementation details, scroll up.

In [ ]:
N = 80000

sim_rolls_df = pd.DataFrame({
    "X_1": np.random.choice(roll_df["x"], size = N, p = roll_df["P(X = x)"]),
    "X_2": np.random.choice(roll_df["x"], size = N, p = roll_df["P(X = x)"])
})

sim_rolls_df

Define the following random variables, which are functions of $X_1$ and $X_2$:
* $Y = X_1 + X_1 = 2 X_1$
* $Z = X_1 + X_2$

We can use our simulated values of $X_1, X_2$ to create new columns $Y$ and $Z$:

In [ ]:
sim_rolls_df['Y'] = 2 * sim_rolls_df['X_1']
sim_rolls_df['Z'] = sim_rolls_df['X_1'] + sim_rolls_df['X_2']
sim_rolls_df

Now that we have simulated samples of $Y$ and $Z$, we can plot histograms to see their distributions!


In [ ]:
px.histogram(sim_rolls_df[["Y", "Z"]].melt(), x="value", color="variable", 
             barmode="overlay", histnorm="probability",
             title="Empirical Distributions")


In [ ]:
pd.DataFrame([
    sim_rolls_df[["Y", "Z"]].mean().rename("Mean"),
    sim_rolls_df[["Y", "Z"]].var().rename("Var"),
    np.sqrt(sim_rolls_df[["Y", "Z"]].var()).rename("SD")
])

<br/><br/>

---

# Which would you pick?

* $\large Y_A = 10 X_1 + 10 X_2 $
* $\large Y_B = \sum\limits_{i=1}^{20} X_i$
* $\large Y_C = 20 X_1$

First let's construct the probability distribution for a single coin. This will let us flip 20 IID coins later.

In [ ]:
# First construct probability distribution for a single fair coin
p = 0.5
coin_df = pd.DataFrame({"x": [1, 0], # [Heads, Tails]
                        "P(X = x)": [p, 1 - p]})
coin_df

## Choice A:
$\large Y_A = 10 X_1 + 10 X_2 $

In [ ]:
N = 10000

np.random.rand(N,2) < p

In [ ]:
sim_flips = pd.DataFrame(
    {"Choice A": np.sum((np.random.rand(N,2) < p) * 10, axis=1)})
sim_flips

## Choice B:

$\large Y_B = \sum\limits_{i=1}^{20} X_i$

In [ ]:
sim_flips["Choice B"] = np.sum((np.random.rand(N,20) < p), axis=1)
sim_flips

## Choice C:

$\large Y_C = 20 X_1$

In [ ]:
sim_flips["Choice C"] = 20  * (np.random.rand(N,1) < p) 
sim_flips

<br/>
If you're curious as to what these distributions look like, I've simulated some populations:

In [ ]:
px.histogram(sim_flips.melt(), x="value", facet_row="variable", 
             barmode="overlay", histnorm="probability",
             title="Empirical Distributions",
             width=600, height=600)

In [ ]:
pd.DataFrame([
    sim_flips.mean().rename("Mean"),
    sim_flips.var().rename("Var"),
    np.sqrt(sim_flips.var()).rename("SD")
])

<br/><br/><br/>

---

# From Population to Sample

Remember the population distribution we looked at earlier:

In [ ]:
dist_df

In [ ]:
# A population generated from the distribution
N = 100000
all_samples = np.random.choice(dist_df["x"], N, p=dist_df["P(X = x)"])
sim_pop_df = pd.DataFrame({"X(s)": all_samples})
sim_pop_df

<br/><br/><br/>
Suppose we draw a sample of size 100 from this giant population.

We are performing **Random Sampling with Replacement:** `df.sample(n, replace=True)` ([link](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html))

In [ ]:
n = 100      # Size of our sample
sample_df = (
             sim_pop_df.sample(n, replace=True)
             # Some reformatting below
             .reset_index(drop=True)
             .rename(columns={"X(s)": "X"})
            )
sample_df

Our **sample distribution** (n = 100):

In [ ]:
px.histogram(sample_df, x="X", histnorm="probability", title="Sample (n = 100)")

<br/>

Compare this to our **original population** (N = 80,000):

In [ ]:
px.histogram(sim_df, x="X(s)", histnorm="probability", title="Population of X")

In [ ]:
pd.DataFrame(
    {"Sample": [sample_df["X"].mean(), sample_df["X"].var(), np.sqrt(sample_df["X"].var())],
     "Population": [sim_df["X(s)"].mean(), sim_df["X(s)"].var(), np.sqrt(sim_df["X(s)"].var())]})